In [ ]:
import tensorflow as tf### models
import numpy as np### math computations
import seaborn as sns### visualizations
import matplotlib.pyplot as plt### plotting bar chart
import datetime
import pathlib
import io
from datetime import datetime
import json
import xml.etree.ElementTree as ET
import os
import shutil
import cv2
import time
import random
from PIL import Image
import albumentations as A
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape, LeakyReLU)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.initializers import RandomNormal

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d huanghanchina/pascal-voc-2012

Dataset URL: https://www.kaggle.com/datasets/huanghanchina/pascal-voc-2012
License(s): DbCL-1.0
100% 3.63G/3.63G [03:06<00:00, 21.5MB/s]
100% 3.63G/3.63G [03:06<00:00, 20.9MB/s]


In [ ]:
val_list=['2007_000027.jpg','2007_000032.jpg','2007_000033.jpg','2007_000039.jpg','2007_000042.jpg','2007_000061.jpg',
          '2007_000063.jpg','2007_000068.jpg','2007_000121.jpg','2007_000123.jpg','2007_000129.jpg','2007_000170.jpg',
          '2007_000175.jpg','2007_000187.jpg','2007_000241.jpg','2007_000243.jpg','2007_000250.jpg','2007_000256.jpg',
          '2007_000272.jpg','2007_000323.jpg','2007_000332.jpg','2007_000333.jpg','2007_000346.jpg','2007_000363.jpg',
          '2007_000364.jpg','2007_000392.jpg','2007_000423.jpg','2007_000452.jpg','2007_000464.jpg','2007_000480.jpg',
          '2007_000491.jpg','2007_000504.jpg','2007_000515.jpg','2007_000528.jpg','2007_000529.jpg','2007_000549.jpg',
          '2007_000559.jpg','2007_000572.jpg','2007_000584.jpg','2007_000629.jpg','2007_000636.jpg','2007_000645.jpg',
          '2007_000648.jpg','2007_000661.jpg','2007_000663.jpg','2007_000664.jpg','2007_000676.jpg','2007_000713.jpg',
          '2007_000720.jpg','2007_000727.jpg','2007_000733.jpg','2007_000738.jpg','2007_000762.jpg','2007_000768.jpg',
          '2007_000783.jpg','2007_000793.jpg','2007_000799.jpg','2007_000804.jpg','2007_000807.jpg','2007_000822.jpg',
          '2007_001299.jpg','2007_001311.jpg','2007_001321.jpg','2007_001340.jpg']

In [ ]:
!unzip "/content/pascal-voc-2012.zip" -d "/content/dataset/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001829.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001874.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001876.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001882.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001885.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001895.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001896.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001926.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001966.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001971.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001992.png  
  inflating: /content/dataset/voc2012/VOC2012/S

In [ ]:
train_images='/content/dataset/VOC2012/JPEGImages/'
train_maps='/content/dataset/VOC2012/Annotations/'

val_images='/content/dataset/VOC2012/ValJPEGImages/'
val_maps='/content/dataset/VOC2012/ValAnnotations/'

classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable',
         'dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

B=2
N_CLASSES=len(classes)
H,W =224,224
SPLIT_SIZE=H//32
print(SPLIT_SIZE)
N_EPOCHS=135
BATCH_SIZE=32

7


In [ ]:
!mkdir /content/dataset/VOC2012/ValJPEGImages/
!mkdir /content/dataset/VOC2012/ValAnnotations/

In [ ]:
for name in val_list:
  shutil.move(train_maps+name[:-3]+"xml", val_maps+name[:-3]+"xml")

In [ ]:
for name in val_list:
  shutil.move(train_images+name, val_images+name)

In [ ]:
def preprocess_xml(filename):
  tree = ET.parse(filename)
  root = tree.getroot()
  size_tree = root.find('size')
  height = float(size_tree.find('height').text)
  width = float(size_tree.find('width').text)
  depth = float(size_tree.find('depth').text)
  bounding_boxes = []
  for object_tree in root.findall('object'):
    for bounding_box in object_tree.iter('bndbox'):
      x_min = float(bounding_box.find('xmin').text)
      x_max = float(bounding_box.find('xmax').text)
      y_min = float(bounding_box.find('ymin').text)
      y_max = float(bounding_box.find('ymax').text)
      break
    class_name = object_tree.find('name').text
    class_dict = {classes[i]:i for i in range(len(classes))}
    bounding_box = [
        (x_min+x_max)/(2*width),(y_min+y_max)/(2*height),(x_max-x_min)/width,
        (y_max-y_min)/height,class_dict[class_name]
    ]
    bounding_boxes.append(bounding_box)
  return tf.convert_to_tensor(bounding_boxes)

In [ ]:
preprocess_xml(train_maps+'2007_000032.xml')

FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset/VOC2012/Annotations/2007_000032.xml'

In [ ]:
def generate_output(bounding_boxes):
  output_label=np.zeros((SPLIT_SIZE,SPLIT_SIZE,N_CLASSES+5))
  for b in range(len(bounding_boxes)):
    grid_x=bounding_boxes[...,b,0]*SPLIT_SIZE
    grid_y=bounding_boxes[...,b,1]*SPLIT_SIZE
    i=int(grid_x)
    j=int(grid_y)

    output_label[i,j,0:5]=[1.,grid_x%1,grid_y%1,bounding_boxes[...,b,2],bounding_boxes[...,b,3]]
    output_label[i,j,5+int(bounding_boxes[...,b,4])]=1.

  return tf.convert_to_tensor(output_label,tf.float32)

In [ ]:
generate_output(preprocess_xml(train_maps+'2007_000032.xml'))

FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset/VOC2012/Annotations/2007_000032.xml'

In [ ]:
im_paths = []
xml_paths = []

val_im_paths = []
val_xml_paths = []

for i in os.listdir(train_maps):
  im_paths.append(train_images+i[:-3]+'jpg')
  xml_paths.append(train_maps+i)

for  i in os.listdir(val_maps):
  val_im_paths.append(val_images+i[:-3]+'jpg')
  val_xml_paths.append(val_maps+i)

print(len(im_paths),len(xml_paths))
print(len(val_im_paths),len(val_xml_paths))

17061 17061
64 64


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((im_paths,xml_paths))
val_dataset = tf.data.Dataset.from_tensor_slices((val_im_paths,val_xml_paths))

In [ ]:
for i in train_dataset.take(1):
  print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'/content/dataset/VOC2012/JPEGImages/2008_006008.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'/content/dataset/VOC2012/Annotations/2008_006008.xml'>)


In [ ]:
def get_imbboxes(im_path,xml_path):
  img=tf.io.decode_jpeg(tf.io.read_file(im_path))
  img=tf.cast(tf.image.resize(img, [H,W]),dtype=tf.float32)

  bboxes=tf.numpy_function(func=preprocess_xml, inp=[xml_path], Tout=tf.float32)
  return img,bboxes

In [ ]:
train_dataset=train_dataset.map(get_imbboxes)
val_dataset=val_dataset.map(get_imbboxes)

In [ ]:
for i,j in train_dataset.skip(50):
  print(i.shape,j)
  break

(224, 224, 3) tf.Tensor([[0.625     0.6291291 0.526     0.4054054 3.       ]], shape=(1, 5), dtype=float32)


In [ ]:
cv2.imwrite('out_1.jpg',np.array(i))

True

In [ ]:
def preprocess(img,bboxes):
  labels=tf.numpy_function(func=generate_output,inp=[bboxes],Tout=(tf.float32))
  return img,labels

In [ ]:
train_dataset=train_dataset.map(preprocess)
val_dataset=val_dataset.map(preprocess)

In [ ]:
train_dataset=(
  train_dataset.
  batch(BATCH_SIZE).
  prefetch(tf.data.AUTOTUNE)
)

In [ ]:
val_dataset=(
  val_dataset.
  batch(BATCH_SIZE).
  prefetch(tf.data.AUTOTUNE)
)

In [ ]:
NUM_FILTERS=512
OUTPUT_DIM=N_CLASSES+5*B

In [ ]:
base_model = tf.keras.applications.resnet50.ResNet50(
# base_model=tf.keras.applications.efficientnet.EfficientNetB1(
    weights='imagenet',
    input_shape=(H,W,3),
    include_top=False,
)
base_model.trainable=False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
# Input layer
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)  # Ensure the base model is in inference mode

# Add custom layers
x = Conv2D(NUM_FILTERS, (3, 3), padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = LeakyReLU(negative_slope=0.1)(x)

x = Conv2D(NUM_FILTERS, (3, 3), padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = LeakyReLU(negative_slope=0.1)(x)

x = Conv2D(NUM_FILTERS, (3, 3), padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = LeakyReLU(negative_slope=0.1)(x)

x = Conv2D(NUM_FILTERS, (3, 3), padding='same', kernel_initializer='he_normal')(x)
x = LeakyReLU(negative_slope=0.1)(x)

x = Flatten()(x)
x = Dense(NUM_FILTERS, kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = LeakyReLU(negative_slope=0.1)(x)

x = Dropout(0.5)(x)
x = Dense(SPLIT_SIZE * SPLIT_SIZE * OUTPUT_DIM, activation='sigmoid')(x)
x = Reshape((SPLIT_SIZE, SPLIT_SIZE, OUTPUT_DIM))(x)

# Define the model
model = Model(inputs=inputs, outputs=x)

# Print model summary
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 7, 7, 512)           │       9,437,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_10 (LeakyReLU)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_11 (LeakyReLU)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_12 (LeakyReLU)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_13 (LeakyReLU)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_14 (LeakyReLU)           │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1470)                │         754,110 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 53,712,702 (204.90 MB)

 Trainable params: 30,120,894 (114.90 MB)

 Non-trainable params: 23,591,808 (90.00 MB)

In [ ]:
def compute_iou(boxes1, boxes2):
    boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,
                         boxes1[..., 1] - boxes1[..., 3] / 2.0,
                         boxes1[..., 0] + boxes1[..., 2] / 2.0,
                         boxes1[..., 1] + boxes1[..., 3] / 2.0],
                        axis=-1)

    boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                         boxes2[..., 1] - boxes2[..., 3] / 2.0,
                         boxes2[..., 0] + boxes2[..., 2] / 2.0,
                         boxes2[..., 1] + boxes2[..., 3] / 2.0],
                        axis=-1)
    lu = tf.maximum(boxes1_t[..., :2], boxes2_t[..., :2])
    rd = tf.minimum(boxes1_t[..., 2:], boxes2_t[..., 2:])

    intersection = tf.maximum(0.0, rd - lu)
    inter_square = intersection[..., 0] * intersection[..., 1]

    square1 = boxes1[..., 2] * boxes1[..., 3]
    square2 = boxes2[..., 2] * boxes2[..., 3]

    union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)
    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)

In [ ]:
def difference(x,y):
  return tf.reduce_sum(tf.square(y-x))

In [ ]:
def yolo_loss(y_true,y_pred):
  #shape of y_true = (none,7,7,25) and shape of target=(none,7,7,1)
  target = y_true=[...,0]

  ####for Object
  y_pred_extract = tf.gather_nd(y_pred,tf.where(target[:]==1))
  y_target_extract = tf.gather_nd(y_true,tf.where(target[:]==1))
  rescaler = tf.where(target[:]==1)*32
  upscaler1 = tf.concat([rescaler[:,1:],tf.zeros([len(rescaler),2],dtype=tf.flaot32)],axis=-1)
  target_upscaler2 = tf.repeat([[32.0,32.0,224.0,224.0]],
                               repeats=[len(rescaler)],axis=0)*tf.cast(y_target_extract[...,1:5],dtype=tf.float32)
  pred1_upscaler2 = tf.repeat([[32.0,32.0,224.0,224.0]],
                               repeats=[len(rescaler)],axis=0)*tf.cast(y_pred_extract[...,1:5],dtype=tf.float32)
  pred2_upscaler2 = tf.repeat([[32.0,32.0,224.0,224.0]],
                               repeats=[len(rescaler)],axis=0)*tf.cast(y_pred_extract[...,6:10],dtype=tf.float32)
  target_orig = tf.cast(upscaler1, dtype = tf.float32)+target_upscaler2
  pred_1_orig = tf.cast(upscaler1, dtype = tf.float32)+pred1_upscaler2
  pred_2_orig = tf.cast(upscaler1, dtype = tf.float32)+pred2_upscaler2
  mask =tf.cast(tf.math.greater(compute_iou(target_orig,pred_1_orig),compute_iou(target_orig,pred_2_orig)),dtype=tf.int32)
  y_pred_joined=tf.transpose(tf.concat([tf.expand_dims(y_pred_extract[...,0],axis=0),
                        tf.expand_dims(y_pred_extract[...,5],axis=0)],axis=0))
  obj_pred = tf.gather_nd(y_pred_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
  obj_loss = difference(tf.cast(obj_pred,dtype=tf.float32),tf.cast(tf.ones([len(rescaler)]),dtype = tf.float32))

  ####for No Object
  y_pred_extract = tf.gather_nd(y_pred[...,0:B*5],tf.where(target[:]==0))
  target_extract = tf.zeros(len(y_pred_extract))
  no_object_loss1 = difference(tf.cast(y_pred_extract[...,0],dtype=tf.float32),tf.cast(target_extract,dtype= tf.float32))
  no_object_loss2 = difference(tf.cast(y_pred_extract[...,5],dtype=tf.float32),tf.cast(target_extract,dtype= tf.float32))
  no_object_loss = no_object_loss1+no_object_loss2

  ######################## For OBject class loss
  y_pred_extract = tf.gather_nd(y_pred[...,10:],tf.where(target[:]==1))
  class_extract = tf.gather_nd(y_true[...,5:],tf.where(target[:]==1))

  class_loss = difference(tf.cast(y_pred_extract,dtype =tf.float32)
                                ,tf.cast(class_extract,dtype=tf.float32))
  ######################### For object bounding box loss
  y_pred_extract = tf.gather_nd(y_pred[...,0:B*5],tf.where(target[:]==1))
  centre_joined = tf.stack([y_pred_extract[...,1:3],y_pred_extract[...,6:8]],axis=1)

  centre_pred = tf.gather_nd(centre_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
  centre_target = tf.gather_nd(y_true[...,1:3],tf.where[target[:]==1])
  centre_loss = difference(centre_pred,centre_target)

  size_joined = tf.stack([y_pred_extract[...,3:5],y_pred_extract[...,8:10]],axis=1)
  size_pred = tf.gather_nd(size_joined,tf.stack([tf.range(len(rescaler)),mask],axis = -1))
  size_target = tf.gather_nd(y_true[...,3:5],tf.where(target[:]==1))
  size_loss = difference(tf.math.sqrt(tf.math.abs(size_pred)),tf.math.sqrt(tf.math.abs(size_target)))
  box_loss = centre_loss + size_loss

  lambda_coord = 5
  lambda_noobj = 0.5
  loss = obj_loss + (lambda_noobj*no_object_loss)+tf.cast(lambda_coord*box_loss,dtype=tf.float32)+ tf.cast(class_loss,dtype=tf.float32)
  return loss


In [ ]:
import tensorflow as tf

def compute_iou(boxes1, boxes2):
    # Compute intersection over union between two sets of boxes
    boxes1 = tf.expand_dims(boxes1, 1)  # (batch, boxes1, 1, 4)
    boxes2 = tf.expand_dims(boxes2, 0)  # (batch, 1, boxes2, 4)

    # Compute intersection
    inter_left_up = tf.maximum(boxes1[..., :2], boxes2[..., :2])
    inter_right_down = tf.minimum(boxes1[..., 2:], boxes2[..., 2:])
    inter_wh = tf.maximum(inter_right_down - inter_left_up, 0.0)
    inter_area = inter_wh[..., 0] * inter_wh[..., 1]

    # Compute union
    box1_area = (boxes1[..., 2] - boxes1[..., 0]) * (boxes1[..., 3] - boxes1[..., 1])
    box2_area = (boxes2[..., 2] - boxes2[..., 0]) * (boxes2[..., 3] - boxes2[..., 1])
    union_area = box1_area + box2_area - inter_area

    # Compute IoU
    return inter_area / (union_area + 1e-8)

def yolo_loss(y_true, y_pred):
    # Shapes assumed: (batch_size, grid_size, grid_size, num_classes + 5)
    # Extract objectness, class, and bounding box predictions and targets

    grid_size = tf.shape(y_true)[1]
    num_classes = tf.shape(y_true)[-1] - 5

    # Reshape predictions and targets
    y_true = tf.reshape(y_true, (-1, grid_size, grid_size, 1, 5 + num_classes))
    y_pred = tf.reshape(y_pred, (-1, grid_size, grid_size, 1, 5 + num_classes))

    # Split predictions
    pred_conf = y_pred[..., 4:5]  # Objectness score
    pred_class = y_pred[..., 5:]  # Class predictions
    pred_boxes = y_pred[..., :4]  # Bounding boxes

    # Split targets
    true_conf = y_true[..., 4:5]
    true_class = y_true[..., 5:]
    true_boxes = y_true[..., :4]

    # Objectness loss
    obj_loss = tf.reduce_sum(tf.square(true_conf - pred_conf))

    # No-object loss
    no_obj_loss = tf.reduce_sum(tf.square((1 - true_conf) * pred_conf))

    # Class loss
    class_loss = tf.reduce_sum(tf.square(true_class - pred_class) * true_conf)

    # Bounding box loss
    box_loss = tf.reduce_sum(tf.square(true_boxes - pred_boxes) * true_conf)

    # Combine losses
    lambda_coord = 5.0  # Weight for bounding box loss
    lambda_noobj = 0.5  # Weight for no-object loss

    loss = obj_loss + lambda_noobj * no_obj_loss + lambda_coord * box_loss + class_loss

    return loss


In [ ]:
checkpoint_filepath='/content/drive/MyDrive/Bang/yolo_resnet50_b1_new.weights.h5'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [ ]:
def scheduler(epoch, lr):
  if epoch < 40:
    return 1e-3
  elif epoch>=40 and epoch<80:
    return 5e-4
  else:
    return 1e-4

In [ ]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
model.compile(
  loss=yolo_loss,
  optimizer=Adam(1e-3),
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.load_weights(checkpoint_filepath)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/Bang/yolo_resnet50_b1_new.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
 model.fit(
  train_dataset,
  validation_data=val_dataset,
  verbose=1,
  epochs=135,
  callbacks = [lr_callback,callback]
)

Epoch 1/135


ValueError: Cannot take the length of shape with unknown rank.

In [ ]:
for images, labels in train_dataset.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)
model_output = model(images)
print("Images shape:", model_output.shape)


Images shape: (32, 224, 224, 3)
Labels shape: (32, 7, 7, 25)
Images shape: (32, 7, 7, 30)
